<a href="https://colab.research.google.com/github/Hamza-Ali0237/HF-Projects/blob/main/Beginner/Sentiment%20Analysis%20App/Sentiment_Analysis_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

In [10]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [11]:
classifier = pipeline(
    task = "text-classification",
    model = model,
    tokenizer = tokenizer
)

Device set to use cuda:0


In [12]:
sample_input = "I liked how the Lakers beat Clippers in the play-offs, I think they are the best!"
output = classifier(sample_input)
print(output)

[{'label': 'LABEL_2', 'score': 0.9879472851753235}]


In [13]:
# Creating a label map to understand the predicted label
label_map = {
    "LABEL_0": "Negative",
    "LABEL_1": "Neutral",
    "LABEL_2": "Positive"
}

In [14]:
# Map label
mapped_output = {
    "label": label_map[output[0]["label"]],
    "score": output[0]["score"]
}

print(mapped_output)

{'label': 'Positive', 'score': 0.9879472851753235}


# Gradio UI

In [15]:
!pip install gradio

In [16]:
import gradio as gr
import matplotlib.pyplot as plt
import pandas as pd

# Function for single tweet
def classify_tweet(tweet):
    if not tweet.strip():
        return "Input is empty", None
    output = classifier(tweet)[0]
    label = label_map[output['label']]
    score = output['score']
    return f"{label} ({score:.2f})", score

# Function for multiple tweets + pie chart
def classify_batch(tweets):
    tweets = [t.strip() for t in tweets.split('\n') if t.strip()]
    results = []
    sentiment_counts = {"Positive": 0, "Neutral": 0, "Negative": 0}

    for tweet in tweets:
        result = classifier(tweet)[0]
        label = label_map[result["label"]]
        score = result["score"]
        sentiment_counts[label] += 1
        results.append({"Tweet": tweet, "Sentiment": label, "Confidence": round(score, 3)})

    # Create pie chart
    fig, ax = plt.subplots()
    ax.pie(sentiment_counts.values(), labels=sentiment_counts.keys(), autopct='%1.1f%%', startangle=90)
    ax.axis('equal')

    return pd.DataFrame(results), fig

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🐦 Twitter Sentiment Analyzer with Hugging Face + Gradio")

    with gr.Tab("Single Tweet"):
        tweet_input = gr.Textbox(label="Enter a tweet", placeholder="e.g. I love using this app!", lines=2)
        output_label = gr.Label(label="Predicted Sentiment")
        score_bar = gr.Slider(minimum=0, maximum=1, label="Confidence Score", interactive=False)
        tweet_button = gr.Button("Analyze")

        tweet_button.click(fn=classify_tweet, inputs=tweet_input, outputs=[output_label, score_bar])

    with gr.Tab("Batch Tweets"):
        batch_input = gr.Textbox(label="Enter one tweet per line", lines=10, placeholder="Tweet 1\nTweet 2\nTweet 3")
        batch_button = gr.Button("Analyze Batch")
        batch_table = gr.Dataframe(headers=["Tweet", "Sentiment", "Confidence"])
        pie_plot = gr.Plot()

        batch_button.click(fn=classify_batch, inputs=batch_input, outputs=[batch_table, pie_plot])

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1eccb8fbd2f91c6f08.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
